## 행정동 법정동 데이터 매핑

In [3]:
import pandas as pd


mapping_bh_dong = pd.read_csv('clean_map_dh_dong.csv', index_col=0)
mapping_bh_dong.head()

,시군구명,행정동코드,행정동명,법정동코드,법정동명,법정동개수
0,종로구,11110515,청운효자동,11110101,청운동,10
1,종로구,11110515,청운효자동,11110102,신교동,10
2,종로구,11110515,청운효자동,11110103,궁정동,10
3,종로구,11110515,청운효자동,11110104,효자동,10
4,종로구,11110515,청운효자동,11110105,창성동,10


## 거주인구

In [4]:
import pandas as pd


live_df = pd.read_csv('거주인구_201911.csv', encoding = 'euc-kr')
live_df

,행정구역(동읍면)별,5세별,항목,단위,2019.11 월,Unnamed: 5
0,서울특별시,계,총인구수[명],명,9732577,NaN
1,서울특별시,계,남자인구수[명],명,4746636,NaN
2,서울특별시,계,여자인구수[명],명,4985941,NaN
3,서울특별시,0 - 4세,총인구수[명],명,302041,NaN
4,서울특별시,0 - 4세,남자인구수[명],명,155294,NaN
...,...,...,...,...,...,...
29695,둔촌제2동,95 - 99세,남자인구수[명],명,2,NaN
29696,둔촌제2동,95 - 99세,여자인구수[명],명,23,NaN
29697,둔촌제2동,100+,총인구수[명],명,14,NaN
29698,둔촌제2동,100+,남자인구수[명],명,1,NaN


In [5]:
# len(live_df['행정구역(동읍면)별'].unique())

#### 전처리

In [6]:
# 행정구역 중 '서울특별시' 제거
# 연령 중 '계' 제거
# 컬럼명 변경
# 인덱스,단위,불필요한 컬럼 제거

live_df = live_df[(live_df['행정구역(동읍면)별']!='서울특별시')&(live_df['5세별']!='계')& (live_df['항목']!='총인구수[명]')]
live_df = live_df.reset_index()
live_df = live_df.rename(columns = {'행정구역(동읍면)별':'행정동', '5세별':'연령대','항목':'성별', '2019.11 월':'거주인구수'})
live_df = live_df.drop(['index','단위','Unnamed: 5'], axis = 1)
live_df

,행정동,연령대,성별,거주인구수
0,종로구,0 - 4세,남자인구수[명],1789
1,종로구,0 - 4세,여자인구수[명],1784
2,종로구,5 - 9세,남자인구수[명],2354
3,종로구,5 - 9세,여자인구수[명],2295
4,종로구,10 - 14세,남자인구수[명],2631
...,...,...,...,...
18853,둔촌제2동,90 - 94세,여자인구수[명],66
18854,둔촌제2동,95 - 99세,남자인구수[명],2
18855,둔촌제2동,95 - 99세,여자인구수[명],23
18856,둔촌제2동,100+,남자인구수[명],1


In [7]:
# live_df['연령대'].unique()

In [8]:
# live_df['성별'].unique()

In [9]:
# 연령대: 5세기준 -> 10세 기준
# 성별 표준화: oo인구수[명] -> oo

live_df = live_df.replace(['0 - 4세','5 - 9세'],'10대 미만', regex = True)
live_df = live_df.replace(['10 - 14세', '15 - 19세'],'10대', regex = True)
live_df = live_df.replace(['20 - 24세', '25 - 29세'],'20대', regex = True)
live_df = live_df.replace(['30 - 34세', '35 - 39세'],'30대', regex = True)
live_df = live_df.replace(['40 - 44세', '45 - 49세'],'40대', regex = True)
live_df = live_df.replace(['50 - 54세', '55 - 59세'],'50대', regex = True)
live_df = live_df.replace(['60 - 64세', '65 - 69세'],'60대', regex = True)
live_df = live_df.replace(['70 - 74세', '75 - 79세','80 - 84세', '85 - 89세', '90 - 94세', '95 - 99세', '100+'],'70대 이상', regex = True)
live_df.loc[live_df['연령대']=='70대 이상+','연령대'] ='70대 이상'

live_df['성별'] = live_df['성별'].apply(lambda x: x[:2])
live_df

,행정동,연령대,성별,거주인구수
0,종로구,10대 미만,남자,1789
1,종로구,10대 미만,여자,1784
2,종로구,10대 미만,남자,2354
3,종로구,10대 미만,여자,2295
4,종로구,10대,남자,2631
...,...,...,...,...
18853,둔촌제2동,70대 이상,여자,66
18854,둔촌제2동,70대 이상,남자,2
18855,둔촌제2동,70대 이상,여자,23
18856,둔촌제2동,70대 이상,남자,1


In [10]:
# live_df['연령대'].unique()

In [11]:
# live_df['성별'].unique()

In [12]:
# len(live_df['행정동'].unique())

In [13]:
#행정동 내 구로 되어 있는 행 삭제
## 행정동 내 value가 지역구인 것은 지역구 컬럼에 작성
## 나머지는 앞의 값에 따라 채우는 걸로
## 인덱스 삭제

gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
        '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
        '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

live_df.loc[live_df['행정동'].isin(gu),'시군구명'] = live_df.loc [live_df['행정동'].isin(gu),'행정동']
live_df = live_df.fillna(method = "ffill")
live_df = live_df[live_df['행정동'].isin(gu) == False].reset_index()
live_df = live_df.drop('index', axis = 1)
live_df

,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,195,종로구
1,청운효자동,10대 미만,여자,193,종로구
2,청운효자동,10대 미만,남자,264,종로구
3,청운효자동,10대 미만,여자,250,종로구
4,청운효자동,10대,남자,306,종로구
...,...,...,...,...,...
17803,둔촌제2동,70대 이상,여자,66,강동구
17804,둔촌제2동,70대 이상,남자,2,강동구
17805,둔촌제2동,70대 이상,여자,23,강동구
17806,둔촌제2동,70대 이상,남자,1,강동구


In [14]:
# live_df['시군구명'].unique()

In [15]:
#지역, 행정동, 성별, 연령별로 인구수 sum

live_df = live_df[['시군구명','행정동','연령대','성별','거주인구수']].groupby(['시군구명','행정동','성별','연령대']).sum().reset_index()
live_df

,시군구명,행정동,성별,연령대,거주인구수
0,강남구,개포1동,남자,10대,302
1,강남구,개포1동,남자,10대 미만,192
2,강남구,개포1동,남자,20대,434
3,강남구,개포1동,남자,30대,344
4,강남구,개포1동,남자,40대,464
...,...,...,...,...,...
6779,중랑구,중화제2동,여자,30대,1847
6780,중랑구,중화제2동,여자,40대,1761
6781,중랑구,중화제2동,여자,50대,2533
6782,중랑구,중화제2동,여자,60대,1967


#### 연도, 월 컬럼 생성 필요

#### 법정동 행정동 매핑

In [16]:
#유동인구데이터프레임과 행정동법정동매핑데이터 합치기

live_df_b = pd.merge(live_df,mapping_bh_dong)[['시군구명','법정동코드','법정동명','법정동개수','성별','연령대','거주인구수']]
live_df_b

,시군구명,법정동코드,법정동명,법정동개수,성별,연령대,거주인구수
0,강남구,11680107,신사동,2,남자,10대,302
1,강남구,11680110,압구정동,2,남자,10대,302
2,강남구,11680108,논현동,1,남자,10대,302
3,강남구,11680108,논현동,1,남자,10대,302
4,강남구,11680107,신사동,2,남자,10대,302
...,...,...,...,...,...,...,...
205467,중랑구,11260104,묵동,1,여자,70대 이상,1808
205468,중랑구,11260105,망우동,1,여자,70대 이상,1808
205469,중랑구,11260105,망우동,1,여자,70대 이상,1808
205470,중랑구,11260106,신내동,1,여자,70대 이상,1808


In [17]:
# live_df_b[live_df_b['법정동명']=='신사동']

In [18]:
live_df_b = live_df_b.groupby(['시군구명','법정동코드','법정동명','성별','연령대','법정동개수']).sum().reset_index()
live_df_b

,시군구명,법정동코드,법정동명,성별,연령대,법정동개수,거주인구수
0,강남구,11680101,역삼동,남자,10대,1,63336
1,강남구,11680101,역삼동,남자,10대 미만,1,39924
2,강남구,11680101,역삼동,남자,20대,1,73092
3,강남구,11680101,역삼동,남자,30대,1,78416
4,강남구,11680101,역삼동,남자,40대,1,92848
...,...,...,...,...,...,...,...
8587,중랑구,11260106,신내동,여자,30대,1,57522
8588,중랑구,11260106,신내동,여자,40대,1,57668
8589,중랑구,11260106,신내동,여자,50대,1,71920
8590,중랑구,11260106,신내동,여자,60대,1,56140


In [19]:
# len(live_df_b['시군구명'].unique())

In [20]:
# len(live_df_b['법정동코드'].unique())

In [21]:
# len(live_df_b['법정동명'].unique())

In [22]:
# len(live_df_b['성별'].unique())

In [23]:
# len(live_df_b['연령대'].unique())

In [24]:
# live_df_b['법정동개수'].unique()

In [25]:
# 거주인구를 법정동 수로 나누기 

live_df_b['거주인구수/n'] = live_df_b['거주인구수']/live_df_b['법정동개수']
live_df_b = live_df_b[['시군구명','법정동코드','법정동명','연령대','거주인구수/n']]
live_df_b

,시군구명,법정동코드,법정동명,연령대,거주인구수/n
0,강남구,11680101,역삼동,10대,63336.0
1,강남구,11680101,역삼동,10대 미만,39924.0
2,강남구,11680101,역삼동,20대,73092.0
3,강남구,11680101,역삼동,30대,78416.0
4,강남구,11680101,역삼동,40대,92848.0
...,...,...,...,...,...
8587,중랑구,11260106,신내동,30대,57522.0
8588,중랑구,11260106,신내동,40대,57668.0
8589,중랑구,11260106,신내동,50대,71920.0
8590,중랑구,11260106,신내동,60대,56140.0


In [26]:
#저장
